# Publish-Subscribe

- Idea
    - You have many different users, and many different event streams
    - Different users want to listen to different streams
        - For example, different Reddit users will want to subscribe to different subreddits
    - How do we manage such a use case?
        - We have a subscription manager (called a `Provider`) to manage the set of users (called `Subscriber`s), and the set of message producers (called `Publisher`s) that write to a `Topic`
        - The Publishers will post a message to a `Topic` contained in the `Provider`
        - The `Provider`, upon seeing a new message, updates all the `Subscribers` of that topic
        - Each `Subscriber` can choose to subscribe or unsubscribe from a topic

In [6]:
class Subscriber:
    def __init__(self, name: str, provider: 'Provider'):
        self.name = name
        self.provider: Provider = provider 

    def subscribe(self, topic: str):
        if topic in self.provider.topic_subscriptions:
            self.provider.topic_subscriptions.get(topic, []).append(self)
        else:
            self.provider.topic_subscriptions[topic] = [self]

    def unsubscribe(self, topic: str):
        if topic in self.provider.topic_subscriptions:
            try:
                self.provider.topic_subscriptions.get(topic, []).remove(self)
            except ValueError:
                pass
    
    def receive_message(self, topic: str, message: str):
        print(f'{self.name} has received {message=} from {topic=}')

class Publisher:
    def __init__(self, name: str, provider: 'Provider'):
        self.name: str = name
        self.provider: Provider = provider

    def post_message(self, message: str):
        if self.name in self.provider.topic_messages:
            self.provider.topic_messages.get(self.name, []).append(message)
        else:
            self.provider.topic_messages[self.name] = [message]
        
        self.provider.update_subscribers(self.name, message)

class Provider:
    def __init__(self):
        self.topic_subscriptions: dict[str, list[Subscriber]] = {}
        self.topic_messages: dict[str, list[str]] = {}

    def update_subscribers(self, topic: str, message: str):
        for subscriber in self.topic_subscriptions.get(topic, []):
            subscriber.receive_message(topic, message)

provider = Provider()

dsa = Publisher('DSA', provider)
cs = Publisher('CS', provider)
skincare = Publisher('skincare', provider)
kpop = Publisher('kpop', provider)
dogs = Publisher('dogs', provider)

yj = Subscriber('YJ', provider)
yj.subscribe('CS')
yj.subscribe('DSA')
yj.subscribe('dogs')

yy = Subscriber('YY', provider)
yy.subscribe('skincare')
yy.subscribe('kpop')
yy.subscribe('dogs')

dsa.post_message('trees video!!!')
dogs.post_message('dogs video!!!')

YJ has received message='trees video!!!' from topic='DSA'
YJ has received message='dogs video!!!' from topic='dogs'
YY has received message='dogs video!!!' from topic='dogs'
